In [1]:
from IPython.display import clear_output
# !wget http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip
# !unzip Market-1501-v15.09.15.zip
clear_output()

In [2]:
import torch
import torchvision.models as models
import torch.nn.functional as F

In [ ]:
class Baseline(torch.nn.Module):
    def __init__(self):
        self.features_model = models.vgg16_bn(pretrained=True).features
        
    def forward(self, anchor, positive, negative):
        # Expected input size as (_, 3, 224, 224)
        # Output size is (_, 512, 7, 7)
        # Resized to (_, 512*7*7)
        anchor = self.features_model(anchor)
        positive = self.features_model(positive)
        negative = self.features_model(negative)
        
        return anchor, positive, negative

def triplet_loss(anchor, positive, negative, margin):
    _, c, h, w = anchor.shape
    anchor = anchor.permute(0, 2, 3, 1).view(-1, h*w, c)
    positive = positive.permute(0, 2, 3, 1).view(-1, h*w, c)
    negative = negative.permute(0, 2, 3, 1).view(-1, h*w, c)
    
    dist_positive = torch.sum(torch.sum((anchor - positive)**2, dim=1)**0.5, dim=1)
    dist_negative = torch.sum(torch.sum((anchor - positive)**2, dim=1)**0.5, dim=1)
    
#     dist_positive = F.pairwise_distance(anchor, positive, 2)
#     dist_negative = F.pairwise_distance(anchor, negative, 2)

    loss = F.relu(dist_positive - dist_negative + self.margin)
    return loss

# def lossless_triplet(anchor, positive, negative, beta=N, epsilon=1e-8):
#     _, c, h, w = anchor.shape
#     anchor = anchor.permute(0, 2, 3, 1).view(-1, c*h*w)
    
#     anchor = F.softmax(anchor, dim=1)
#     positive = F.softmax(positive, dim=1)
#     negative = F.softmax(negative, dim=1)
    
    
#     loss = 
#     pass

In [ ]:
# VGG input size (_, 3, 224, 224)
print(model.features)

In [9]:
import numpy as np
torch.sum(torch.from_numpy(np.arange(1, 19).reshape(2, 3, 3)), dim=1)

tensor([[12, 15, 18],
        [39, 42, 45]])

In [27]:
A, B = np.arange(1, 19).reshape(2, 3, 3), np.arange(19, 1, -1).reshape(2, 3, 3)
A = np.moveaxis(A, [0, 1, 2], [1, 2, 0])
B = np.moveaxis(B, [0, 1, 2], [1, 2, 0])

print(torch.sum(torch.from_numpy((A - B).reshape(6, 3)), dim=1))

tensor([-36,  18, -30,  24, -24,  30])


<h1>Prepare Market-1501 dataset for triplet-loss<h1>

In [4]:
import os
import cv2
import numpy as np

trainDir = "/home/bone/share/Re-id/Market-1501-v15.09.15/gt_bbox"
fileNames = os.listdir(trainDir)
fileDict = {}
personIDs = []
for file in fileNames:
    personID = file[0:4]
    if personID in fileDict:
        fileDict[personID].append(file)
    else:
        fileDict[personID] = [file]
        personIDs.append(personID)

In [9]:
from numpy import random
from tqdm import tqdm

def readFromGtBBox(name):
    image = cv2.imread(trainDir + '/' + name)
    ## Transform here
    return image

dataset = []
dataframeAmount = 300
for i in tqdm(range(dataframeAmount)):
    subject = random.randint(0,len(personIDs))
    subjectID = personIDs[subject]
    negativeSubject = random.randint(0,len(personIDs))
    negativeSubjectID = personIDs[negativeSubject]
    anchor = fileDict[subjectID][random.randint(0,len(fileDict[subjectID]))]
    positive = fileDict[subjectID][random.randint(0,len(fileDict[subjectID]))]
    negative = fileDict[negativeSubjectID][random.randint(0,len(fileDict[negativeSubjectID]))]
    dataset.append([readFromGtBBox(anchor),readFromGtBBox(positive),readFromGtBBox(negative)])

dataset = np.asarray(dataset)
print('Done loading dataset. Shape: ', dataset.shape)

100%|██████████| 300/300 [00:02<00:00, 122.43it/s]

Done loading dataset. Shape:  (300, 3)


In [22]:
trainPortion = 0.8
trainSet = dataset[0:int(dataset.shape[0] * trainPortion),:]
testSet = dataset[int(dataset.shape[0] * trainPortion) :,:]

In [23]:
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.utils.data as data

def transformer(image):
    image = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.406, 0.456), (0.229, 0.225, 0.224)),
        #transforms.RandomHorizontalFlip(p=0.5),
    ])(image)
    return image

In [27]:
## Utilize the downloaded dataset ##
class MarketTripletLoader(data.Dataset):
    def __init__(self,train=True, transform=None):
        self.transform = transform
        if train:
            imageShape = dataset[0,0].shape
            self.marketImages = trainSet
        else:
            self.marketImages = testSet
      
    def __getitem__(self, idx):
        anchor = self.marketImages[idx,0,:]
        positive = self.marketImages[idx,1,:]
        negative = self.marketImages[idx,2,:]
        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negtive = self.transform(negative)
        return anchor, positive, negative
    
    def __len__(self):
        return self.marketImages.shape[0]

In [28]:
marketTrain = MarketTripletLoader(transform = transformer)
marketTest = MarketTripletLoader(train = False, transform = transformer)
marketTrainLoader = data.DataLoader(dataset=marketTrain, batch_size=128,num_workers=4,shuffle=True)
marketTestLoader = data.DataLoader(dataset=marketTest, batch_size=128,num_workers=4,shuffle=False)